# Analyzing Hacker News Posts
Hacker News is a site started by the startup incubator [Y Combinator](https://www.ycombinator.com/), where user-submitted stories (known as "posts") receive votes and comments, similar to reddit. Hacker News is extremely popular in technology and startup circles, and posts that make it to the top of the Hacker News listings can get hundreds of thousands of visitors as a result.

For this miniproject, we will use a downsampled [dataset](https://dq-content.s3.amazonaws.com/356/hacker_news.csv) from the Hacker News website, which consists of approximately 20000 random samples of submissions that received at least 1 comment.

Below are descriptions of the columns:
* id: the unique identifier from Hacker News for the post
* title: the title of the post
* url: the URL that the posts links to, if the post has a URL
* num_points: the number of points the post acquired, calculated as the total number of upvotes minus the total number of downvotes
* num_comments: the number of comments on the post
* author: the username of the person who submitted the post
* created_at: the date and time of the post's submission

We're specifically interested in posts with titles that begin with either "Ask HN" or "Show HN". Users submit "Ask HN" posts to ask the Hacker News community a specific question. Likewise, users submit "Show HN" posts to show the Hacker News community a project, product, or just something interesting.

We'll compare these two types of posts to determine the following:

1) Do Ask HN or Show HN receive more comments on average?
2) Do posts created at a certain time receive more comments on average?

In [1]:
import csv

with open("hacker_news.csv", "r", encoding="utf8") as hn_data:
    hn = list(csv.reader(hn_data))
    
for i in range(5):
    print(hn[i])

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']
['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52']
['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30']
['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20']
['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01']


In [2]:
# Storing the headers in a separate variable before deleting them from the dataset for easier processing

headers = hn[0]

del hn[0]

print(headers)
print()
for i in range(5):
    print(hn[i])

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']

['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52']
['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30']
['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20']
['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01']
['10301696', 'Note by Note: The Making of Steinway L1037 (2007)', 'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0', '8', '2', 'walterbell', '9/30/2015 4:12']


In [3]:
# Extracting the specific entries desired - "Ask HN" and "Show HN"

ask_posts = []
show_posts = []
other_posts = []

for post in hn:
    title = post[1]
    
    if title.lower().startswith("ask hn"):
        ask_posts.append(post)
    elif title.lower().startswith("show hn"):
        show_posts.append(post)
    else:
        other_posts.append(post)

print("Total posts:", len(hn))
print("Number of Ask HN posts:", len(ask_posts))
print("Number of Show HN posts:", len(show_posts))
print("Number of non-Ask, non-Show HN posts:", len(other_posts))

Total posts: 20100
Number of Ask HN posts: 1744
Number of Show HN posts: 1162
Number of non-Ask, non-Show HN posts: 17194


In [4]:
# Calculates the number of comments on a posts dataset
def calc_avg_comments(dataset, index):
    total_comments = 0
    
    for post in dataset:
        total_comments += int(post[index])
        
    return total_comments/len(dataset)

avg_ask_comments = calc_avg_comments(ask_posts, 4)
print("Average number of comments in Ask HN posts:", avg_ask_comments)
avg_show_comments = calc_avg_comments(show_posts, 4)
print("Average number of comments in Show HN posts:", avg_show_comments)



Average number of comments in Ask HN posts: 14.038417431192661
Average number of comments in Show HN posts: 10.31669535283993


"Ask HN" posts receive more comments on average!

Since "Ask HN" posts are more likely to receive comments, we'll focus our remaining analysis just on these posts.

Next, we'll determine if "Ask HN" posts created at a certain time are more likely to attract comments. We'll use the following steps to perform this analysis:

1) Calculate the number of "Ask HN" posts created in each hour of the day, along with the number of comments received.
2) Calculate the average number of comments "Ask HN" posts receive by hour created.

In [11]:
# Calculating the number of ask posts and comments by hour created

import datetime as dt

result_list = []

for post in ask_posts:
    result_list.append([post[6], int(post[4])])
    
counts_by_hour = {}
comments_by_hour = {}

for result in result_list:
    pdt = dt.datetime.strptime(result[0], "%m/%d/%Y %H:%M")
    hour = pdt.strftime("%H")
    
    if hour not in counts_by_hour:
        counts_by_hour[hour] = 1
        comments_by_hour[hour] = result[1]
    else:
        counts_by_hour[hour] += 1
        comments_by_hour[hour] += result[1]

counts_by_hour = dict(sorted(counts_by_hour.items()))
comments_by_hour = dict(sorted(comments_by_hour.items()))
print(counts_by_hour) 
print(comments_by_hour)


# Calculates the average number of comments per post for each hour of the day

avg_by_hour = []

for hour in counts_by_hour:
    if hour in comments_by_hour:
        avg_by_hour.append([hour, comments_by_hour[hour]/counts_by_hour[hour]])

print()
print(avg_by_hour)


# Sorts and arranges the average comments per post in descending order

swap_avg_by_hour = []

for item in avg_by_hour:
    swap_avg_by_hour.append([item[1], item[0]])

sorted_swap = sorted(swap_avg_by_hour, reverse=True)


# Prints top 5 hours for Ask HN posts comments

print()
print("Top 5 Hours for Ask Posts Comments")

for i in range(5):
    tdt = dt.datetime.strptime(sorted_swap[i][1], "%H")
    print(f"{tdt.strftime("%H:%M")}: {sorted_swap[i][0]:.2f} average comments per post")



{'00': 55, '01': 60, '02': 58, '03': 54, '04': 47, '05': 46, '06': 44, '07': 34, '08': 48, '09': 45, '10': 59, '11': 58, '12': 73, '13': 85, '14': 107, '15': 116, '16': 108, '17': 100, '18': 109, '19': 110, '20': 80, '21': 109, '22': 71, '23': 68}
{'00': 447, '01': 683, '02': 1381, '03': 421, '04': 337, '05': 464, '06': 397, '07': 267, '08': 492, '09': 251, '10': 793, '11': 641, '12': 687, '13': 1253, '14': 1416, '15': 4477, '16': 1814, '17': 1146, '18': 1439, '19': 1188, '20': 1722, '21': 1745, '22': 479, '23': 543}

[['00', 8.127272727272727], ['01', 11.383333333333333], ['02', 23.810344827586206], ['03', 7.796296296296297], ['04', 7.170212765957447], ['05', 10.08695652173913], ['06', 9.022727272727273], ['07', 7.852941176470588], ['08', 10.25], ['09', 5.5777777777777775], ['10', 13.440677966101696], ['11', 11.051724137931034], ['12', 9.41095890410959], ['13', 14.741176470588234], ['14', 13.233644859813085], ['15', 38.5948275862069], ['16', 16.796296296296298], ['17', 11.46], ['18', 

# Conclusions
* The way to create a post with the highest chances of receiving comments is by crafting an "Ask HN" post in the periods between 15-16 (3-4 PM) and 20-21 (8-9 PM).
* All times are US EST times according to the dataset's documentation.
* There is an outlier at 2 AM as well.